In [ ]:
import os

import numpy as np
import xarray as xr
import xrspatial
from xrspatial.terrain import generate_terrain
import rioxarray
import datashader as ds
import rasterio
from rasterio.io import MemoryFile
from rasterio.enums import Resampling
from azure.storage.blob import BlobClient
from azure.storage.blob import ContentSettings

In [ ]:
# example with terrain and add 1 band
cvs = ds.Canvas(x_range=(-20,20), y_range=(-20,20), plot_height=400, plot_width=800)
terrain_a = generate_terrain(canvas=cvs)
terrain_a.data = terrain_a.data.astype(np.int32)
terrain_a = terrain_a.assign_coords(band = np.asarray(1, np.int32))
terrain_a = terrain_a.expand_dims('band')
terrain_b = generate_terrain(canvas=cvs, seed=1)
terrain_b.data = terrain_b.data.astype(np.int32)
terrain_b = terrain_b.assign_coords(band = np.asarray(2, np.int32))
terrain_b = terrain_b.expand_dims('band')
terrain = xr.concat([terrain_a, terrain_b], 'band')
terrain['band']

In [ ]:
def set_get_blob(filename):
    blob = BlobClient("https://xarrayspatial.blob.core.windows.net/", 
                      "examples-data", 
                      f"{filename}.tiff",
                     credential=os.environ['AZURE_ACCESS_KEY'])
    return blob

In [ ]:
y_size = terrain.data.shape[1]
x_size = terrain.data.shape[2]
num_bands = len(terrain['band'])

with MemoryFile() as memfile:
    with memfile.open(driver='GTiff',
                      width=x_size, height=y_size,
                      count=num_bands,
                      dtype=rasterio.dtypes.int32) as dst:
        dst.write(terrain)
        dst.build_overviews([2], Resampling.nearest)  
    blob = set_get_blob("terrain_ovws_1")
    blob.upload_blob(memfile)
    content_settings = ContentSettings(content_type='image/tiff')
    blob.set_http_headers(content_settings=content_settings)